# Space Invaders

## Dependencies

In [1]:
!pip install tensorflow==2.9.1 gym keras-rl2 gym[atari]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 511.7 MB 5.0 kB/s 
     |████████████████████████████████| 52 kB 858 kB/s 
     |████████████████████████████████| 5.8 MB 38.3 MB/s 
     |████████████████████████████████| 438 kB 18.8 MB/s 
     |████████████████████████████████| 1.6 MB 40.8 MB/s 
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.8.0
    Uninstalling tensorflow-estimator-2.8.0:
      Successfully uninstalled tensorflow-estimator-2.8.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.8.0
    Uninstalling tensorboard-2.8.0:
      Successfully uninstalled tensorboard-2.8.0
  Attempting uninstall: keras
    Found existing installation: keras 2.8.0
    Uninstalling keras-2.8.0:
      Successfully uninstalled keras-2.8.0
  Attempting uninstall: gast
    Found existing installation: gast 0.5.3
    Uninsta

ROM instructions: https://github.com/openai/atari-py#roms

In [2]:
!python -m atari_py.import_roms roms

copying space_invaders.bin from roms/Space Invaders.bin to /usr/local/lib/python3.7/dist-packages/atari_py/atari_roms/space_invaders.bin


## Exploration and baseline

In [3]:
import gym
import random
import numpy as np

In [49]:
env = gym.make("SpaceInvaders-v4")
print(env.observation_space.shape)

(210, 160, 3)


In [50]:
env.unwrapped.get_action_meanings()

['NOOP', 'FIRE', 'RIGHT', 'LEFT', 'RIGHTFIRE', 'LEFTFIRE']

In [52]:
from numpy import clip

EPISODES = 100
scores = []

for episode in range(1, EPISODES + 1):
    state = env.reset()
    done = False
    score = 0 
    
    while not done:
        # env.render()
        action = random.choice(range(env.action_space.n))
        n_state, reward, done, info = env.step(action)
        score += clip(reward, -1.0, 1.0)
    
    scores.append(score)
    print(f"Episode {episode}: Reward == {score}")

avg = np.mean(scores)
print(f"Average reward: {avg}")
env.close()

Episode 1: Reward == 10.0
Episode 2: Reward == 9.0
Episode 3: Reward == 7.0
Episode 4: Reward == 9.0
Episode 5: Reward == 8.0
Episode 6: Reward == 6.0
Episode 7: Reward == 10.0
Episode 8: Reward == 4.0
Episode 9: Reward == 9.0
Episode 10: Reward == 6.0
Episode 11: Reward == 13.0
Episode 12: Reward == 11.0
Episode 13: Reward == 10.0
Episode 14: Reward == 12.0
Episode 15: Reward == 12.0
Episode 16: Reward == 5.0
Episode 17: Reward == 14.0
Episode 18: Reward == 9.0
Episode 19: Reward == 5.0
Episode 20: Reward == 12.0
Episode 21: Reward == 9.0
Episode 22: Reward == 9.0
Episode 23: Reward == 12.0
Episode 24: Reward == 10.0
Episode 25: Reward == 5.0
Episode 26: Reward == 5.0
Episode 27: Reward == 4.0
Episode 28: Reward == 25.0
Episode 29: Reward == 19.0
Episode 30: Reward == 12.0
Episode 31: Reward == 24.0
Episode 32: Reward == 12.0
Episode 33: Reward == 9.0
Episode 34: Reward == 12.0
Episode 35: Reward == 10.0
Episode 36: Reward == 10.0
Episode 37: Reward == 9.0
Episode 38: Reward == 9.0
Ep

So the baseline is around 150 unclipped / 9 clipped.

## Model

In [4]:
from tensorflow.keras import Input
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Convolution2D, Resizing, Rescaling, Reshape
from tensorflow.keras.optimizers import Adam
from tensorflow.image import rgb_to_grayscale
from tensorflow.keras.layers import Layer
from tensorflow.keras.utils import register_keras_serializable

In [ ]:
@register_keras_serializable("atari")
class GrayscaleLayer(Layer):
  def call(self, input):
    return rgb_to_grayscale(input)


In [ ]:
def build_model(window_size, height, width, channels, actions):
    model = Sequential()
    model.add(Input(shape=(window_size, height, width, channels)))
    model.add(Reshape((window_size * height, width, channels), name="reshape_stack"))
    model.add(GrayscaleLayer(name="grayscale"))
    model.add(Resizing((window_size * height) // 2, width // 2, name="resize_half"))
    model.add(Rescaling(1./255, name="normalize")) # normalize to [0, 1]
    model.add(Reshape((window_size, height // 2, width // 2, 1), name="reshape_unstack"))
    model.add(Convolution2D(32, (8,8), strides=(4,4), activation='relu', name="conv1"))
    model.add(Convolution2D(64, (4,4), strides=(2,2), activation='relu', name="conv2"))
    model.add(Convolution2D(64, (3,3), activation='relu', name="conv3"))
    model.add(Flatten(name="flatten"))
    model.add(Dense(512, activation='relu', name="fully_connected_1"))
    # model.add(Dense(256, activation='relu'))
    model.add(Dense(actions, activation='linear', name="output"))
    return model

In [ ]:
WINDOW_SIZE = 4
height, width, channels = env.observation_space.shape
actions = env.action_space.n

In [ ]:
model = build_model(WINDOW_SIZE, height, width, channels, actions)

In [ ]:
model.summary()

Model: "seqmodel"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 reshape (Reshape)           (None, 840, 160, 3)       0         
                                                                 
 grayscale (GrayscaleLayer)  (None, 840, 160, 1)       0         
                                                                 
 resizing (Resizing)         (None, 420, 80, 1)        0         
                                                                 
 rescaling (Rescaling)       (None, 420, 80, 1)        0         
                                                                 
 reshape_1 (Reshape)         (None, 4, 105, 80, 1)     0         
                                                                 
 conv2d (Conv2D)             (None, 4, 25, 19, 32)     2080      
                                                                 
 conv2d_1 (Conv2D)           (None, 4, 11, 8, 64)      328

## Agent

In [27]:
from rl.agents import DQNAgent
from rl.memory import SequentialMemory
from rl.policy import LinearAnnealedPolicy, EpsGreedyQPolicy
from rl.processors import Processor
from numpy import clip

In [28]:
class AtariRewardProcessor(Processor):
  def process_reward(self, reward):
      return clip(reward, -1.0, 1.0)

In [41]:
def build_agent(model, actions, window_size):
    policy = LinearAnnealedPolicy(
        EpsGreedyQPolicy(), 
        attr='eps', 
        value_max=1.0, 
        value_min=0.1, 
        value_test=0.2, 
        nb_steps=75000
    )
    memory = SequentialMemory(
        limit=1000000, 
        window_length=window_size
    )
    dqn = DQNAgent(
        model=model, 
        memory=memory, 
        policy=policy,
        processor=AtariRewardProcessor(),
        enable_dueling_network=True, 
        dueling_type='avg', 
        nb_actions=actions, 
        nb_steps_warmup=10000,
        gamma=0.99
    )
    return dqn

In [42]:
dqn = build_agent(model, actions, WINDOW_SIZE)
dqn.compile(Adam(learning_rate=0.00025))

NameError: ignored

## Train

In [ ]:
dqn.fit(env, nb_steps=10000, visualize=False, verbose=2)

Training for 10000 steps ...


/usr/local/lib/python3.7/dist-packages/keras/engine/training_v1.py:2067: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


  545/10000: episode: 1, duration: 16.783s, episode steps: 545, steps per second:  32, episode reward: 120.000, mean reward:  0.220 [ 0.000, 30.000], mean action: 2.461 [0.000, 5.000],  loss: --, mean_q: --, mean_eps: --


/usr/local/lib/python3.7/dist-packages/keras/engine/training_v1.py:2067: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


 1906/10000: episode: 2, duration: 1463.794s, episode steps: 1361, steps per second:   1, episode reward: 605.000, mean reward:  0.445 [ 0.000, 200.000], mean action: 2.497 [0.000, 5.000],  loss: 8.339929, mean_q: 0.921901, mean_eps: 0.869230
 2575/10000: episode: 3, duration: 1071.301s, episode steps: 669, steps per second:   1, episode reward: 110.000, mean reward:  0.164 [ 0.000, 25.000], mean action: 2.469 [0.000, 5.000],  loss: 14.594663, mean_q: 1.724796, mean_eps: 0.798400
 4166/10000: episode: 4, duration: 2526.268s, episode steps: 1591, steps per second:   1, episode reward: 250.000, mean reward:  0.157 [ 0.000, 30.000], mean action: 2.456 [0.000, 5.000],  loss: 2.485551, mean_q: 0.938030, mean_eps: 0.696700
 4569/10000: episode: 5, duration: 642.676s, episode steps: 403, steps per second:   1, episode reward: 55.000, mean reward:  0.136 [ 0.000, 20.000], mean action: 2.546 [0.000, 5.000],  loss: 0.923119, mean_q: 0.454591, mean_eps: 0.606970
 5166/10000: episode: 6, duration:

In [ ]:
scores = dqn.test(env, nb_episodes=20, visualize=False)
np.mean(scores.history["episode_reward"])

Testing for 20 episodes ...
Episode 1: reward: 40.000, steps: 694
Episode 2: reward: 230.000, steps: 826
Episode 3: reward: 25.000, steps: 570
Episode 4: reward: 60.000, steps: 792
Episode 5: reward: 35.000, steps: 557
Episode 6: reward: 20.000, steps: 661
Episode 7: reward: 225.000, steps: 967
Episode 8: reward: 115.000, steps: 844
Episode 9: reward: 80.000, steps: 677
Episode 10: reward: 85.000, steps: 545
Episode 11: reward: 75.000, steps: 672
Episode 12: reward: 90.000, steps: 1360
Episode 13: reward: 50.000, steps: 375
Episode 14: reward: 105.000, steps: 1188
Episode 15: reward: 80.000, steps: 683
Episode 16: reward: 15.000, steps: 497
Episode 17: reward: 340.000, steps: 988
Episode 18: reward: 10.000, steps: 734
Episode 19: reward: 65.000, steps: 435
Episode 20: reward: 20.000, steps: 407


88.25

TODO: 
* Train with 1.5M~2M steps
* Add fourth conv layer
* Don't use grayscale layer but initialize the environment with the grayscale option instead

## RAM-based approach

Atari 2600 uses a 128 byte RAM to for its internal representation of the game state.

In [31]:
envram = gym.make("SpaceInvaders-v4", obs_type="ram")

In [32]:
envram.observation_space.shape

(128,)

In [33]:
def build_ram_model(window_size, ram_size, actions):
    model = Sequential(name="ram_model")
    model.add(Input(shape=(window_size, ram_size)))
    model.add(Flatten(name="flatten"))
    model.add(Dense(512, activation="relu", name="fc1"))
    model.add(Dense(128, activation="relu", name="fc2"))
    model.add(Dense(actions, activation="linear", name="output"))
    return model


In [36]:
WINDOW_SIZE = 3
ram_model = build_ram_model(WINDOW_SIZE, envram.observation_space.shape[0], envram.action_space.n)

In [37]:
ram_model.summary()

Model: "ram_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 384)               0         
                                                                 
 fc1 (Dense)                 (None, 512)               197120    
                                                                 
 fc2 (Dense)                 (None, 128)               65664     
                                                                 
 output (Dense)              (None, 6)                 774       
                                                                 
Total params: 263,558
Trainable params: 263,558
Non-trainable params: 0
_________________________________________________________________


In [43]:
ram_dqn = build_agent(ram_model, envram.action_space.n, WINDOW_SIZE)

In [44]:
ram_dqn.compile(Adam(learning_rate=0.0002))

In [45]:
history = ram_dqn.fit(envram, nb_steps=100_000, visualize=False, verbose=2)

Training for 100000 steps ...


/usr/local/lib/python3.7/dist-packages/keras/engine/training_v1.py:2067: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


   995/100000: episode: 1, duration: 2.017s, episode steps: 995, steps per second: 493, episode reward: 20.000, mean reward:  0.020 [ 0.000,  1.000], mean action: 2.519 [0.000, 5.000],  loss: --, mean_q: --, mean_eps: --
  1744/100000: episode: 2, duration: 1.395s, episode steps: 749, steps per second: 537, episode reward:  9.000, mean reward:  0.012 [ 0.000,  1.000], mean action: 2.401 [0.000, 5.000],  loss: --, mean_q: --, mean_eps: --
  2420/100000: episode: 3, duration: 1.263s, episode steps: 676, steps per second: 535, episode reward:  9.000, mean reward:  0.013 [ 0.000,  1.000], mean action: 2.635 [0.000, 5.000],  loss: --, mean_q: --, mean_eps: --
  3217/100000: episode: 4, duration: 1.467s, episode steps: 797, steps per second: 543, episode reward: 12.000, mean reward:  0.015 [ 0.000,  1.000], mean action: 2.353 [0.000, 5.000],  loss: --, mean_q: --, mean_eps: --
  3693/100000: episode: 5, duration: 0.866s, episode steps: 476, steps per second: 550, episode reward:  4.000, mean

/usr/local/lib/python3.7/dist-packages/keras/engine/training_v1.py:2067: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


 10122/100000: episode: 13, duration: 3.754s, episode steps: 724, steps per second: 193, episode reward: 10.000, mean reward:  0.014 [ 0.000,  1.000], mean action: 2.330 [0.000, 5.000],  loss: 195.357807, mean_q: 75.091032, mean_eps: 0.879268
 10854/100000: episode: 14, duration: 10.153s, episode steps: 732, steps per second:  72, episode reward:  9.000, mean reward:  0.012 [ 0.000,  1.000], mean action: 2.551 [0.000, 5.000],  loss: 65.992800, mean_q: 65.533273, mean_eps: 0.874150
 11619/100000: episode: 15, duration: 10.630s, episode steps: 765, steps per second:  72, episode reward: 12.000, mean reward:  0.016 [ 0.000,  1.000], mean action: 2.561 [0.000, 5.000],  loss: 51.122107, mean_q: 63.545574, mean_eps: 0.865168
 12231/100000: episode: 16, duration: 8.476s, episode steps: 612, steps per second:  72, episode reward:  7.000, mean reward:  0.011 [ 0.000,  1.000], mean action: 2.552 [0.000, 5.000],  loss: 44.517604, mean_q: 62.400450, mean_eps: 0.856906
 13168/100000: episode: 17, d

In [48]:
scores = ram_dqn.test(envram, nb_episodes=100, visualize=False)
np.mean(scores.history["episode_reward"])

Testing for 100 episodes ...
Episode 1: reward: 10.000, steps: 811
Episode 2: reward: 16.000, steps: 968
Episode 3: reward: 20.000, steps: 1055
Episode 4: reward: 12.000, steps: 848
Episode 5: reward: 12.000, steps: 842
Episode 6: reward: 18.000, steps: 1070
Episode 7: reward: 16.000, steps: 967
Episode 8: reward: 12.000, steps: 809
Episode 9: reward: 12.000, steps: 840
Episode 10: reward: 9.000, steps: 650
Episode 11: reward: 12.000, steps: 852
Episode 12: reward: 16.000, steps: 951
Episode 13: reward: 10.000, steps: 785
Episode 14: reward: 12.000, steps: 842
Episode 15: reward: 12.000, steps: 835
Episode 16: reward: 12.000, steps: 807
Episode 17: reward: 17.000, steps: 955
Episode 18: reward: 12.000, steps: 810
Episode 19: reward: 11.000, steps: 813
Episode 20: reward: 12.000, steps: 844
Episode 21: reward: 11.000, steps: 807
Episode 22: reward: 12.000, steps: 832
Episode 23: reward: 12.000, steps: 858
Episode 24: reward: 15.000, steps: 953
Episode 25: reward: 12.000, steps: 808
Epis

12.8

In [47]:
ram_dqn.save_weights("ramdqn.h5f", overwrite=True)